In [7]:
from ultralytics import YOLO
import cv2
import os

# 1) Charger le modèle entraîné sur LostAndFound


model_path = "/home/imadb/runs/detect/LostAndFound/weights/best.pt"
assert os.path.exists(model_path), model_path

model = YOLO(model_path)
#  change le chemin si ton modèle LostAndFound est ailleurs

# 2) Image à tester (une image du dossier test)
img_path = "leftImg8bit/test/15_Rechbergstr_Deckenpfronn/15_Rechbergstr_Deckenpfronn_000000_000180_leftImg8bit.png"
#  adapte le nom si besoin

# Vérification
assert os.path.exists(img_path), f"Image non trouvée : {img_path}"

# 3) Inference (YOLO ne dessine rien automatiquement ici)
results = model(img_path, conf=0.25, verbose=False)[0]

print("=== Détections ===")
if len(results.boxes) == 0:
    print("Aucun obstacle détecté")

for box in results.boxes:
    cls = int(box.cls)
    name = results.names[cls]
    conf = float(box.conf)
    print(f"- {name} ({conf:.2f})")

# 4) Charger l’image originale
img = cv2.imread(img_path)

# 5) Dessiner les bounding boxes
for box in results.boxes:
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
    conf = float(box.conf)

    label = f"obstacle {conf:.2f}"
    color = (0, 0, 255)  # rouge (danger)

    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        img,
        label,
        (x1, max(y1 - 10, 20)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        color,
        2,
        cv2.LINE_AA
    )

# 6) Sauvegarder le résultat
out_path = "result_lost_and_found.jpg"
cv2.imwrite(out_path, img)

print("Image sauvegardée :", out_path)


=== Détections ===
- obstacle (0.94)
Image sauvegardée : result_lost_and_found.jpg
